In [ ]:
import gym
from env import Env

env = gym.make(Env.MOUNTAINCAR.value)
print(dir(env))
print(f'{env._max_episode_steps=}')
print(f'{env.action_space=}')
print(f'{env.metadata=}')
print(f'{env.observation_space.shape[0]=}')
print(f'{env.reward_range=}')
print(f'{env.seed=}')
print(f'{env.spec=}')

In [ ]:
## dqn cart-pole 학습

from env import Env
from runner import RunnerParams
from algorithms.dqn import DQNParams
from algorithms.dqn_runner import DQNRunner

runner_param = RunnerParams(record_baseline=499, reward_scale=100.0, max_video=30)
algo_param = DQNParams()
DQNRunner(Env.CARTPOLE.value, algo_param, runner_param).run()

In [ ]:
## ddqn lunar-lander 불러오기

from env import Env
from runner import RunnerParams
from algorithms.dqn import DQNParams
from algorithms.dqn_runner import DQNRunner

runner_param = RunnerParams(load_net=True, load_name='node512-score165-DQN-LunarLander-v2-1631737560.pt',
                            train=False,
                            target_score=150, record_baseline=149, 
                            reward_scale=30.0, max_video=100, video_record_interval=1,
                            step_wrapper=lambda x: (x[0], x[1], x[2], x[3]))
algo_param = DQNParams(n_node=512, batch_size=32, buffer_limit=100000, 
                        n_train_start=8000, start_epsilon=0.2,
                        update_interval=40)
DQNRunner(Env.LUNARLANDER.value, algo_param, runner_param).run()

In [ ]:
## ddqn lunar-lander
from env import Env
from runner import RunnerParams
from algorithms.ddqn import DDQNParams
from algorithms.ddqn_runner import DDQNRunner

runner_param = RunnerParams(save_net=True,
                            train=True,
                            target_score=200, record_baseline=250, 
                            reward_scale=30.0, max_video=100, video_record_interval=200,
                            step_wrapper=lambda x: x)
algo_param = DDQNParams(n_node=512, batch_size=64, buffer_limit=50000, 
                        n_train_start=4000, start_epsilon=0.2,
                        update_interval=20)
DDQNRunner(Env.LUNARLANDER.value, algo_param, runner_param).run()

In [1]:
## ddqn lunar-lander 불러오기

from env import Env
from runner import RunnerParams
from algorithms.ddqn import DDQNParams
from algorithms.ddqn_runner import DDQNRunner

runner_param = RunnerParams(save_net=False, load_net=True, load_name='node512-score213-DDQN-LunarLander-v2-1631750946.pt',
                            train=False,
                            target_score=200, record_baseline=250, 
                            reward_scale=30.0, max_video=100, video_record_interval=1,
                            step_wrapper=lambda x: x)
algo_param = DDQNParams(n_node=512, batch_size=64, buffer_limit=50000, 
                        n_train_start=4000, start_epsilon=0.2,
                        update_interval=20)
DDQNRunner(Env.LUNARLANDER.value, algo_param, runner_param).run()

초기 설정 시작
algorithm: DDQN
env: LunarLander-v2
state space: (8,)
action space: Discrete(4)
네트워크 로딩 시작
시뮬레이션 시작
n_epi=1, aver_score=-16.366906086492445 비디오 저장


C:\Users\kuro1\anaconda3\envs\py38-pytorch-gpu\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] 스레드 모드가 설정된 후에는 바꿀 수 없습니다
  warnings.warn(str(err))


n_epi=2, aver_score=95.91624584171439 비디오 저장
n_epi=3, aver_score=142.20829278904475 비디오 저장
n_epi=4, aver_score=151.07765804515364 비디오 저장
n_epi=5, aver_score=116.72424474090158 비디오 저장
n_epi=6, aver_score=120.21745259779796 비디오 저장
n_epi=7, aver_score=134.25531758905572 비디오 저장
n_epi=8, aver_score=95.17483255827392 비디오 저장
n_epi=9, aver_score=76.11423156411335 비디오 저장
n_epi=10, aver_score=84.90391378626535 비디오 저장
n_epi=11, aver_score=99.12806587723867 비디오 저장
n_epi=12, aver_score=90.42583730574016 비디오 저장
n_epi=13, aver_score=78.87080070098547 비디오 저장
n_epi=14, aver_score=83.98909106479992 비디오 저장
n_epi=15, aver_score=92.05173493034516 비디오 저장
n_epi=16, aver_score=100.01793935201447 비디오 저장
n_epi=17, aver_score=91.78921444096933 비디오 저장
n_epi=18, aver_score=89.14252367037741 비디오 저장
n_epi=19, aver_score=90.3187751993134 비디오 저장
n_epi=20, aver_score=95.47480541188145 비디오 저장
# of episode: 20, avg score: 95.5
n_buffer : 0, eps : 0.0%
n_epi=21, aver_score=95.75972848647025 비디오 저장
n_epi=22, aver_score=114

KeyboardInterrupt: 

In [ ]:
# 알고리즘 테스트 

# ppo

# from tester import RunnerTester
# from algorithms.ppo_runner import PPORunner
# from algorithms.ppo import PPOParams
# from env import Env
# algo_params = PPOParams()
# result = RunnerTester(PPORunner, algo_params, [Env.MOUNTAINCAR]).test()
# print('통과' if result else '실패')


# ppolstm

# from tester import RunnerTester
# from algorithms.ppolstm_runner import PPOlstmRunner
# from algorithms.ppolstm import PPOlstmParams
# from env import Env
# algo_params = PPOlstmParams()
# result = RunnerTester(PPOlstmRunner, algo_params, [Env.MOUNTAINCAR]).test()
# print('통과' if result else '실패')


# actorcritic

# from tester import RunnerTester
# from algorithms.actorcritic_runner import ActorCriticRunner
# from algorithms.actorcritic import ActorCriticParams
# from env import Env
# algo_params = ActorCriticParams()
# result = RunnerTester(ActorCriticRunner, algo_params, [Env.CARTPOLE]).test()
# print('통과' if result else '실패')



# dqn
# from tester import RunnerTester
# from algorithms.dqn_runner import DQNRunner
# from algorithms.dqn import DQNParams
# from env import Env
# algo_params = DQNParams()
# result = RunnerTester(DQNRunner, algo_params, [Env.CARTPOLE]).test()
# print('통과' if result else '실패')



# ddqn
from tester import RunnerTester
from algorithms.ddqn_runner import DDQNRunner
from algorithms.ddqn import DDQNParams
from env import Env
algo_params = DDQNParams()
result = RunnerTester(DDQNRunner, algo_params, [Env.CARTPOLE]).test()
print('통과' if result else '실패')

In [3]:
import os

home_path = os.path.expanduser('~')
cur_path = os.getcwd()
conda_path = home_path + "\\anaconda3"
conda_script_path = home_path + "\\anaconda3\\Scripts\\activate.bat"
exc = ' '.join(['start', '%windir%\System32\cmd.exe "/K"', conda_script_path, conda_path])
!$exc

'conda activate py38-pytorch-gpu && tensorboard --port=6006 --logdir=runs'
'http://localhost:6006/'


'http://localhost:6006/'